# Problem 1
We can write a generic CRT solver that can, up to the limits of the chosen integer type (signed 64-bit integer in my implementation), iteratively solve a system of linear congruences:

$$
\begin{aligned}
x &= a_0 \mod m_0 \\
x &= a_1 \mod m_1 \\
&\ldots \\
x &= a_{n-1} \mod m_{n-1}
\end{aligned}
$$

Solving a system of 1 congruence is trivial, since we can simply return $a_0$. Now that we have a base case, we can inductively build up the solution to the entire system: suppose we have a solution to the first $k$ congruences, denoted by $c_k$, then we can find a solution to the first $k+1$ congruences by making two observations.

First, $c_{k+1}$ must also be a solution to the first $k$ congruences, meaning that for some integer $y$:

$$
\begin{equation}
c_{k+1} = c_k + ( \Pi_{i=0}^{k-1}m_k ) \cdot y
\end{equation}
$$

Second, $c_{k+1}$ must also satisfy the $k+1$'s congruence, meaning that:

$$
\begin{equation}
c_{k+1} \equiv a_k \mod m_k
\end{equation}
$$

Combining the two equations above we have:

$$
c_k + ( \Pi_{i=0}^{k-1}m_k ) \cdot y \equiv a_k \mod m_k
$$

Which can be easily transformed to find a value of $y$:

$$
y \equiv ( \Pi_{i=0}^{k-1}m_k )^{-1} \cdot (a_k - c_k) \mod m_k
$$

After that, we can plug $y$ back into the first obervsation and obtain a solution to the first $k+1$'s congruences.

The CRT solver's source code can be found at the end of this write up.

## a)
The solution is 31

## b)
The solution is 5764

## c)
The solution is 221

## d)
Note that all modulo arithmetics are well-defined, since addition, subtraction, and multiplication are always well-defined, and the multiplicative inverse is well-defined iff the two modulos are relatively prime. However, when the two modulos are not relatively prime, it is possible that the inverse does not exist. For example, the element $2 \in \mathbb{Z}_4$ has no inverse. We can use this two construct impossible congruences such as:

$$
\begin{aligned}
x &\equiv 1 \mod 2 \\
x &\equiv 2 \mod 4 \\
\end{aligned}
$$

This system is impossible because the first congruence requires $x$ to be odd, but the second one requires $x$ to be even.

## Appendix
Source code for the CRT solver (written in Rust, btw):

```rust
/// subtract y from x within the input modulo
pub fn modulo_sub(x: i64, y: i64, modulo: i64) -> i64 {
    let x = x % modulo;
    let y = y % modulo;

    if x - y >= 0 {
        return x - y;
    }
    return x - y + modulo;
}

/// Returns (gcd, s, t) such that s*x + t*y = gcd is the Bezout identity
pub fn extended_gcd(x: i64, y: i64) -> (i64, i64, i64) {
    let (mut prev_r, mut r) = (x, y);
    let (mut prev_s, mut s) = (1, 0);
    let (mut prev_t, mut t) = (0, 1);

    while r != 0 {
        let q = prev_r / r;
        (prev_r, r) = (r, prev_r - q * r);
        (prev_s, s) = (s, prev_s - q * s);
        (prev_t, t) = (t, prev_t - q * t);
    }

    return (prev_r, prev_s, prev_t);
}

/// Attempt to find a multiplicative inverse of x (mod y). This is possible iff
/// x and y are relatively prime. If no multiplicative inverse if possible,
/// return None
pub fn modulo_invert(x: i64, y: i64) -> Option<i64> {
    let (gcd, s, _t) = extended_gcd(x, y);
    if gcd > 1 {
        return None;
    }
    return Some(s);
}

#[derive(Debug)]
pub struct CRT {
    /// The modulo up to which the solution is unique. In the context of Chinese
    /// remainder theorem, it is the product of all modulos in all congruences
    modulo: i64,

    /// The solution to the set of congruences, unique up to self.modulo.
    sol: Option<i64>,
}

impl CRT {
    pub fn new() -> Self {
        return Self {
            sol: None,
            modulo: 1,
        };
    }

    /// Update the internal state to solve the union of the existing system and
    /// the input congruence. Return the solution after the update
    pub fn add_congruence(
        &mut self,
        remainder: i64,
        modulo: i64,
    ) -> Option<i64> {
        if self.sol.is_none() {
            self.sol = Some(remainder);
            self.modulo = modulo;
        } else {
            let mut sol = self.sol.unwrap();
            let diff = modulo_sub(remainder, sol, modulo);
            let inverse = modulo_invert(self.modulo, modulo).unwrap();
            sol = sol + self.modulo * diff * inverse;
            self.modulo = self.modulo * modulo;
            self.sol = Some(modulo_sub(sol, 0, self.modulo));
        }
        return self.sol;
    }
}
```

<p style="page-break-after:always;"></p>

# Problem 2

## a)
We will prove the main statement that showing that if there is one root, then there is necessarily exactly one other root.

Suppose that for odd prime $p$ and some $b$ such that $p$ does not divide $b$, we have already found one square root $x$ such that $x ^ 2 \equiv b \mod p$. If there also exists some $y$ such that $y^2 \equiv b \mod p$, then we can subtract the two equations and get the following relation:

$$
y^2 - x^2 \equiv 0 \mod p
$$

The left hand side of the equation can be factored into $(y+x)(y-x)$, meaning that $y$ is either the additive inverse of $x$ (aka $p-x$) or $x$ itself, thus we either have no square roots or exactly two square roots.

When **$p$ is 2**, we can trivially show that whichever $x$ we choose, $p - x$ is exactly $x$ ($2 - 0 \equiv 0$, $2 - 1 \equiv 1$), so we don't have distinct roots.

When **$p$ divides $b$**, the right hand side of $x^2 \equiv b \mod p$ becomes $0$, so we have exactly one distinct square root of $0$.

## b)
Since the numbers are sufficiently small, a brute-force implementation is used to find square roots:

```rust
pub fn sqrt(x: i64, modulo: i64) -> Vec<i64> {
    return (0..modulo).filter_map(|elem| {
        if modulo_sub(elem * elem, x, modulo) == 0 {
            return Some(elem);
        }
        return None;
    })
    .collect();
}
```

So we have the following results:

- The square roots of 2 mod 7 are 3 and 4
- The square roots of 5 mod 11 are 4 and 7
- There is no square root of 7 mod 11
- The square roots of 3 mod 37 are 15 and 22

## c)
29 has four square roots mod 35 (8, 13, 22, 27). This does not contradict part a because $35 = 5 \cdot 7$ is not a prime number.

## d)
If $a \equiv g^k \mod p$ has square roots, then there exists $x \in \mathbb{Z}_p$ such that $x^2 \equiv a \mod p$. Because $g$ is a generator of the multiplicative group $\mathbb{Z}_p$, there also exists integer $l$ such that $x \equiv g^l \mod p$. Substitute the latter relation into the former, we have:

$$
\begin{aligned}
a &\equiv x^2 \mod p \\
&\equiv (g^l)^2 \mod p \\
&\equiv g^{2l} \mod p \\
&\equiv g^k \mod p
\end{aligned}
$$

Therefore $k$ is necessarily even.

On the other hand, if $k$ is even, then $k = 2 \cdot l$ for some integer l, and we can trivially find $g^l$ to be a square root of $a$ since $(g^l)^2 = g^{2l} = g^k \equiv a \mod p$

# Problem 3

## a)
If there are more than $d$ distinct roots for some polynomial of degree $d$ in $\mathbb{F}$, then denote them as $x_1, x_2, \ldots, x_n$, and we can write the polynomial as the following form:

$$
p(x) = a \cdot \Pi_{i}^n (x - x_i)
$$

where $a \in \mathbb{F}$ is a leading coefficient of the polynomial. However, the right hand side of the equation above is actually a polynomial of degree $n > d$, hence a contradiction happens when a degree-d polynomial has more than d distinct roots.

## b)
Recall from the definition of the order of a element $g \in \mathbb{F}$ to be the smallest exponent $k > 1$ such that $g^k \equiv 1$. Thus for some exponent $e$ and primitive root $g \in \mathbb{F}_{13}$, the order of $g^e$ is the smallest $k > 1$ such that $(g^e)^k \equiv 1$.

We already know that the order of the primitive root $g$ is 12, which means that $(g^e)^k \equiv g^{12}$, so $k$ is the smallest integer $k > 1$ such that $e ^ k$ is a multiple of $12$. From here we can deduce that for some exponent $e$, the order of $g^e$ is the following:

$$
\begin{aligned}
k &= \frac{\text{lcm}(e, 12)}{e} \\
&= \frac{\frac{e \cdot 12}{\text{gcd}(e, 12)}}{e} \\
&= \frac{12}{\text{gcd}(e, 12)}
\end{aligned}
$$

Plugging in the appropriate values of $e$ we can show that for $e \in \{1, 5, 7, 11\}$, the order of $g^e$ is 12. By the definition of a primitive root, then, $g^1, g^5, g^7, g^11$ are all primitive elements of $\mathbb{F}_{13}$

## c)
We can reuse the formula developed in part b (except for when $e = 0$, though in this case the order is trivially 1):

|exponent|order|
|:--|:--|
|0|1|
|1|12|
|2|6|
|3|4|
|4|3|
|5|12|
|6|2|
|7|12|
|8|3|
|9|4|
|10|6|
|11|12|
|12|1|

## d)
From proposition 1.29 we know that since $p = 13$ is a prime number and $a^{p-1} \equiv 1 \mod p$ for all $a$ that are relatively prime with $p$. This means that the only possible orders of elements are integers that divide 12: 1, 2, 3, 4, 6, 12. Since there is no primitive root, the only possible orders are 1, 2, 3, 4, 6.

...